In [1]:
# loading modules
import ee
import geemap
import time
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# initialize ee 
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score


# import all FMSE py functions 
from FMSE import *


In [11]:

# Define a region of interest (ROI).
roi = ee.Geometry.Point([-113.5911, 37.0855]).buffer(10000)

# Load the NLCD 2016 image.
class_image = ee.Image('USGS/NLCD/NLCD2016').select('landcover')



# Method 1: Grouped reducer with pixel area sum
start_time = time.time()
class_areas_sum = ee.Image.pixelArea().addBands(class_image) \
  .reduceRegion(
    reducer=ee.Reducer.sum().group(
      groupField=1,
      groupName='code'
    ),
    geometry=roi,
    scale=30,  # sample the geometry at 30m intervals
    maxPixels=1e10
  ).get('groups')
end_time = time.time()
method_1_time = end_time - start_time

# Method 2: Frequency histogram
start_time = time.time()
class_areas_histogram = class_image.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=roi,
    scale=30,
    maxPixels=1e10
).get('landcover')
end_time = time.time()
method_2_time = end_time - start_time

# Convert pixel areas to square kilometers for Method 1
def convert_area_sum_to_sq_km(grouped_data):
    area_km2 = {}
    for group in grouped_data:
        code = int(group['code'])
        area = group['sum'] / 1e6  # convert from m2 to km2
        area_km2[code] = area
    return area_km2

# Convert pixel count to area in square kilometers for Method 2
def convert_histogram_to_sq_km(histogram_data):
    pixel_area = 30 * 30  # each pixel is 30m x 30m
    area_km2 = {int(k): v * pixel_area / 1e6 for k, v in histogram_data.items()}  # convert from m2 to km2
    return area_km2

# Fetch and convert the data
class_areas_sum = class_areas_sum.getInfo()
class_areas_histogram = class_areas_histogram.getInfo()

# Convert the results to square kilometers
area_sum_km2 = convert_area_sum_to_sq_km(class_areas_sum)
area_histogram_km2 = convert_histogram_to_sq_km(class_areas_histogram)

# Print results
print('Method 1 - Grouped Reducer with Pixel Area Sum (sq km):', area_sum_km2)
print('Time taken by Method 1:', method_1_time, 'seconds')
print('Method 2 - Frequency Histogram (sq km):', area_histogram_km2)
print('Time taken by Method 2:', method_2_time, 'seconds')

# Create a pandas DataFrame for comparison
df = pd.DataFrame({
    'Land Cover Class': area_sum_km2.keys(),
    'Method 1 - Area (sq km)': area_sum_km2.values(),
    'Method 2 - Area (sq km)': [area_histogram_km2.get(k, 0) for k in area_sum_km2.keys()]
})

# Save the DataFrame as a CSV file
#df.to_csv('land_cover_area_comparison.csv', index=False)

print(df)



Method 1 - Grouped Reducer with Pixel Area Sum (sq km): {11: 0.10178568634033203, 21: 26.962950906954898, 22: 35.987747757852, 23: 21.058694187710625, 24: 4.962861159863521, 31: 0.438736302734375, 41: 0.03366579455566406, 42: 0.44562988670869713, 43: 0.4068410438232422, 52: 202.69140692464111, 71: 4.821215176034966, 81: 0.7517330098266601, 82: 5.747426923278809, 90: 5.6541578453091486, 95: 0.4617137600978477}
Time taken by Method 1: 0.0 seconds
Method 2 - Frequency Histogram (sq km): {11: 0.10347529411764705, 21: 26.996544705882346, 22: 35.97126000000001, 23: 21.051716470588236, 24: 4.934325882352941, 31: 0.4329, 41: 0.0351, 42: 0.44544705882352936, 43: 0.4077, 52: 202.7164658823528, 71: 4.7998658823529405, 81: 0.7479, 82: 5.7564, 90: 5.661984705882352, 95: 0.4651094117647058}
Time taken by Method 2: 0.0 seconds
    Land Cover Class  Method 1 - Area (sq km)  Method 2 - Area (sq km)
0                 11                 0.101786                 0.103475
1                 21              